# Run Scheduling Shift
Example (Shift Scheduling at a Retail Store).  
Consider a retail store that is open for business from 10:00 to 21:00.  
There are five shift patterns.  

| Pattern |  Hours to work | Total hours | Cost |
|:-------:|:--------------:|:-----------:|:----:|
|    1    | 10:00 to 18:00 |      8      |  50  |
|    2    | 13:00 to 21:00 |      8      |  60  |
|    3    | 12:00 to 18:00 |      6      |  30  |
|    4    | 10:00 to 13:00 |      3      |  15  |
|    5    | 18:00 to 21:00 |      3      |  13  |  

Staffing requirements at the store vary from hour to hour.  
  
|  Period  | Staffing Requirement |
|:--------:|:--------------------:|
| 10 to 11 |           3          |
| 11 to 12 |           4          |
| 12 to 13 |           6          |
| 13 to 14 |           4          |
| 14 to 15 |           7          |
| 15 to 16 |           8          |
| 16 to 17 |           7          |
| 17 to 18 |           6          |
| 18 to 19 |           4          |
| 19 to 20 |           7          |
| 20 to 21 |           8          |

In [1]:
# libraries
import pandas as pd
from model import Shifts

## Sets & Parameters

In [2]:
df_shifts = pd.read_csv('shifts.csv')
df_forecast = pd.read_csv('forecast.csv')
sets = {}
parameters = {}

sets['Forecast'] = df_forecast['staffing'].to_list
sets['Shifts'] = df_shifts['pattern'].to_list()

parameters['Cost'] = df_shifts['cost'].to_list()

The problem can be formulated as an integer program with the following $c$
vector, $A$ matrix and $b$ vector:  
$$ c = (50,60,30,15,15) $$ 
$$ A = \begin{bmatrix}
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 1 & 1 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 1 \\

\end{bmatrix} \quad 
b = \begin{bmatrix}
3 \\
4 \\
6 \\
4 \\
7 \\
8 \\
7 \\
6 \\
4 \\
7 \\
8 \\

\end{bmatrix}

## Create $A$ Matrix with shift & forecast data 

In [3]:
def create_shift_pattern(shifts: pd.DataFrame, forecast: pd.DataFrame):
    """Create Matrix A  shift coverage pattern"""
    A = [[0 for _ in range(len(shifts))] for __ in range(len(forecast))]
    for j, shift in shifts.iterrows():
        for i, demand in forecast.iterrows():
            if shift['period_in'] <= i < shift['period_out']:
                A[i][j] = 1
    return A

hour_period = {valor: clave for clave, valor in df_forecast['period'].to_dict().items()}
df_shifts['period_in'] = df_shifts['hour_init'].apply(lambda x: hour_period[x])
df_shifts['period_out'] = df_shifts['hour_end'].apply(lambda x: hour_period[x-1])


parameters['A'] = create_shift_pattern(df_shifts, df_forecast)

## Create Model

In [6]:
model = Shifts()
model.sets(sets)
model.parameters(parameters)
model.variables()
model.constraints()
model.objective()
result = model.execute()


An error occurred while loading the CBC library:	 cannot load library '/Users/cristobalcordova/Documents/Shift-Scheduling/venv/lib/python3.11/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib': dlopen(/Users/cristobalcordova/Documents/Shift-Scheduling/venv/lib/python3.11/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib, 0x0002): tried: '/Users/cristobalcordova/Documents/Shift-Scheduling/venv/lib/python3.11/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/cristobalcordova/Documents/Shift-Scheduling/venv/lib/python3.11/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (no such file), '/Users/cristobalcordova/Documents/Shift-Scheduling/venv/lib/python3.11/site-packages/mip/libraries/cbc-c-darwin-x86-64.dylib' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')).  Additionally, ctypes.util.find_library() did not manage

NameError: name 'cbclib' is not defined

In [5]:
import os
os.environ['DYLD_LIBRARY_PATH'] = '/Users/cristobalcordova/dist/lib'